# Limpieza y preparación de datos - Ecobici

## Introducción

Este conjunto de datos fue obtenido de [Ecobici - Datos Abiertos](https://ecobici.cdmx.gob.mx/datos-abiertos/), contiene información histórica sobre los viajes realizados durante el mes de agosto de 2024.

Las columnas que se encuentran son las siguientes:

*   Genero_Usuario (Masculino = M, Femenino = F, Otro/prefiero no decir = O, ? )
*   Edad_Usuario
*   Bici
*   Ciclo_Estacion_Retiro
*   Fecha_Retiro
*   Hora_Retiro
*   Ciclo_EstacionArribo
*   Fecha Arribo
*   Hora_Arribo

## Objetivos

**Identificación y manejo de valores faltantes**

- Detectar campos con valores nulos o vacíos
- Decidir estrategia para cada tipo de valor faltante (eliminación, imputación, etc.)

**Corrección de inconsistencias**

- Verificar y corregir cualquier inconsistencia

**Estandarización de formatos**

- Unificar el formato de fechas (si aplica)
- Normalizar los nombres de las columnas para facilitar el análisis

**Verificación de integridad de datos**

- Validar que los registros se encuentren en el rango de fecha correspondiente al mes de agosto 2024
- Verificar que todas las ciclo estaciones tengan asignado un cluster, tanto de retiro como de arribo
- Verificar que se cuente con ciclo estaciones de retiro/arribo válidas

**Creación de variables**

- Calcular la duración del viaje en minutos
- Generar variables categóricas basadas en la edad de usuarios ('-18' | '18-23' | '24-34', etc)
- Generar variables temporales: hora, día de la semana (número), día de la semana (nombre), aplica para retiros/arribos

**Validación de datos limpios**


## Dependencias

In [1]:
import pandas as pd

## Cargando datos

El archivo original está en formato **CSV**

### Viajes

In [2]:
viajes_path = "2024-08.csv"
df_viajes = pd.read_csv(viajes_path)

#### Visualización incial

In [3]:
df_viajes.head(2)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha_Arribo,Hora_Arribo
0,M,51.0,8620775,017,31/07/2024,23:53:06,061,01/08/2024,00:00:00
1,M,35.0,8882193,335,31/07/2024,23:46:43,320,01/08/2024,00:00:03


In [4]:
numero_viajes_inicial = len(df_viajes)
print(f'Número de viajes inicial: {numero_viajes_inicial:,} ')

Número de viajes inicial: 1,891,319 


### Ciclo estaciones

In [5]:
ciclo_estaciones_path = "ciclo_estaciones_cluster.csv"

df_ciclo_estaciones =pd.read_csv(ciclo_estaciones_path)

#### Visualización inicial

In [6]:
df_ciclo_estaciones.head(2)

,station_id,external_id,name,short_name,lat,lon,rental_methods,capacity,electric_bike_surcharge_waiver,is_charging,eightd_has_key_dispenser,has_kiosk,cluster
0,1,e961269c-34c4-4b70-8e30-a51aa95a8429,CE-710 Molino del Rey - Glorieta de la Lealtad,710,19.416795,-99.192508,"['CREDITCARD', 'KEY']",39,False,False,False,True,5
1,5,3ea89109-d2f3-46eb-9c41-c43742050340,CE-407 Prolongación Xochicalco-General Emilia...,407,19.367266,-99.158656,"['CREDITCARD', 'KEY']",19,False,False,False,True,2


## Información general

## Viajes

### Información General

In [7]:
# info
df_viajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1891319 entries, 0 to 1891318
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Genero_Usuario         object 
 1   Edad_Usuario           float64
 2   Bici                   int64  
 3   Ciclo_Estacion_Retiro  object 
 4   Fecha_Retiro           object 
 5   Hora_Retiro            object 
 6   Ciclo_EstacionArribo   object 
 7   Fecha_Arribo           object 
 8   Hora_Arribo            object 
dtypes: float64(1), int64(1), object(7)
memory usage: 129.9+ MB


In [8]:
# Describe

df_viajes.describe()

,Edad_Usuario,Bici
count,1.891298e+06,1.891319e+06
mean,3.362021e+01,5.457336e+06
std,9.664720e+00,2.032810e+06
min,1.600000e+01,2.000461e+06
25%,2.700000e+01,3.708237e+06
50%,3.200000e+01,5.456421e+06
75%,3.800000e+01,7.220593e+06
max,1.040000e+02,8.999708e+06


In [9]:
# count
df_viajes.count()

Genero_Usuario           1891304
Edad_Usuario             1891298
Bici                     1891319
Ciclo_Estacion_Retiro    1891319
Fecha_Retiro             1891319
Hora_Retiro              1891319
Ciclo_EstacionArribo     1891319
Fecha_Arribo             1891319
Hora_Arribo              1891319
dtype: int64

In [10]:
# Nulos
df_viajes.isnull().sum()

Genero_Usuario           15
Edad_Usuario             21
Bici                      0
Ciclo_Estacion_Retiro     0
Fecha_Retiro              0
Hora_Retiro               0
Ciclo_EstacionArribo      0
Fecha_Arribo              0
Hora_Arribo               0
dtype: int64

### Edades

In [11]:
edad_max = df_viajes['Edad_Usuario'].max()
edad_min = df_viajes['Edad_Usuario'].min()
edad_promedio = df_viajes['Edad_Usuario'].mean()
edad_moda = df_viajes['Edad_Usuario'].mode()
edad_mediana = df_viajes['Edad_Usuario'].median()

print(f"Edad máxima: {edad_max} | Edad mínima: {edad_min} | Edad promedio: {edad_promedio} | Edad mediana: {edad_mediana}")

Edad máxima: 104.0 | Edad mínima: 16.0 | Edad promedio: 33.620210035647474 | Edad mediana: 32.0


In [12]:
# Edades - Moda
edad_moda

0    30.0
Name: Edad_Usuario, dtype: float64

## Limpieza datos

### Viajes

#### Añadiendo columnas - 'Fecha_Retiro_DateTime' | 'Fecha_Arribo_DateTime'

In [13]:
# 'Fecha_Retiro_DateTime'
df_viajes['Fecha_Retiro_DateTime'] = pd.to_datetime(df_viajes['Fecha_Retiro'] + ' ' + df_viajes['Hora_Retiro'], format='%d/%m/%Y %H:%M:%S')

# 'Fecha_Arribo_DateTime'
df_viajes['Fecha_Arribo_DateTime'] = pd.to_datetime(df_viajes['Fecha_Arribo'] + ' ' + df_viajes['Hora_Arribo'], format='%d/%m/%Y %H:%M:%S')


In [14]:
df_viajes.head(2)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,Fecha_Arribo_DateTime
0,M,51.0,8620775,017,31/07/2024,23:53:06,061,01/08/2024,00:00:00,2024-07-31 23:53:06,2024-08-01 00:00:00
1,M,35.0,8882193,335,31/07/2024,23:46:43,320,01/08/2024,00:00:03,2024-07-31 23:46:43,2024-08-01 00:00:03


#### Renombrando Columnas

In [15]:
df_viajes = df_viajes.rename(columns={'Ciclo_EstacionArribo': 'Ciclo_Estacion_Arribo'})

df_viajes.head(1)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,Fecha_Arribo_DateTime
0,M,51.0,8620775,017,31/07/2024,23:53:06,061,01/08/2024,00:00:00,2024-07-31 23:53:06,2024-08-01


#### Validando rango de fecha 01/08/2024 - 31/08/2024

In [16]:
df_viajes = df_viajes[df_viajes['Fecha_Retiro_DateTime'].between('2024-08-01', '2024-08-31')]
df_viajes.head(1)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,Fecha_Arribo_DateTime
44,O,53.0,2199032,331,01/08/2024,00:01:50,312,01/08/2024,00:04:26,2024-08-01 00:01:50,2024-08-01 00:04:26


#### Añadiendo columna 'Grupo_Edad'

In [17]:
# Definiendo bins y etiquetas

bins = [0, 18, 24, 35, 45, 55, 65, 75, 85, 95, 105]
labels = ['-18', '18-23', '24-34', '35-44', '45-54', '55-64', '65-74', '75-84', '85-94', '95+']

In [18]:
df_viajes['Grupo_Edad'] = pd.cut(df_viajes['Edad_Usuario'], bins=bins, labels=labels, right=False)

df_viajes.head(2)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,Fecha_Arribo_DateTime,Grupo_Edad
44,O,53.0,2199032,331,01/08/2024,00:01:50,312,01/08/2024,00:04:26,2024-08-01 00:01:50,2024-08-01 00:04:26,45-54
51,F,20.0,3530268,104,01/08/2024,00:00:06,055,01/08/2024,00:05:00,2024-08-01 00:00:06,2024-08-01 00:05:00,18-23


#### Añadiendo columna 'Duracion_viaje' | 'Duracion_viaje_minutos'

In [19]:
# Función para convertir minutos decimal a formato legible en string

def decimal_minutes_to_string(decimal_minutes):
  minutes = int(decimal_minutes)
  seconds = int((decimal_minutes - minutes) * 60)
  return f"{minutes:02d} minutos {seconds:02d} segundos"

In [20]:
'''
Duracion_viaje - string (formato legible)
Duracion_viaje_minutos - decimal
'''

df_viajes['Duracion_viaje_minutos'] = (df_viajes['Fecha_Arribo_DateTime'] - df_viajes['Fecha_Retiro_DateTime']).dt.total_seconds() / 60
df_viajes['Duracion_viaje'] = df_viajes['Duracion_viaje_minutos'].apply(decimal_minutes_to_string)

df_viajes.head(2)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,Fecha_Arribo_DateTime,Grupo_Edad,Duracion_viaje_minutos,Duracion_viaje
44,O,53.0,2199032,331,01/08/2024,00:01:50,312,01/08/2024,00:04:26,2024-08-01 00:01:50,2024-08-01 00:04:26,45-54,2.6,02 minutos 36 segundos
51,F,20.0,3530268,104,01/08/2024,00:00:06,055,01/08/2024,00:05:00,2024-08-01 00:00:06,2024-08-01 00:05:00,18-23,4.9,04 minutos 54 segundos


#### Cambiando '?' por 'O' en 'Genero_Usuario'

In [21]:
# Genero_Usuario con valor '?' | 'O'
genero_usuario_interrogacion = (df_viajes['Genero_Usuario'] == '?').sum()
genero_usuario_o = (df_viajes['Genero_Usuario'] == 'O').sum()
print(f'Número de registros con Genero_Usuario "?": {genero_usuario_interrogacion} ')
print(f'Número de registros con Genero_Usuario "O": {genero_usuario_o} ')

Número de registros con Genero_Usuario "?": 19 
Número de registros con Genero_Usuario "O": 36008 


In [22]:
# Reemplazar '?' por 'O'
df_viajes['Genero_Usuario'] = df_viajes['Genero_Usuario'].replace('?', 'O')

# Genero_Usuario con valor '?' | 'O'
genero_usuario_interrogacion = (df_viajes['Genero_Usuario'] == '?').sum()
genero_usuario_o = (df_viajes['Genero_Usuario'] == 'O').sum()
print(f'Número de registros con Genero_Usuario "?": {genero_usuario_interrogacion} ')
print(f'Número de registros con Genero_Usuario "O": {genero_usuario_o} ')

Número de registros con Genero_Usuario "?": 0 
Número de registros con Genero_Usuario "O": 36027 


### Añadieno datos temporales para ciclo estaciones de retiro/arribo

In [23]:
# Datos temporales: Ciclo estaciones de retiro
df_viajes['Hora_Retiro_Digito'] = df_viajes['Fecha_Retiro_DateTime'].dt.hour
df_viajes['Dia_Semana_Retiro'] = df_viajes['Fecha_Retiro_DateTime'].dt.day_name()
df_viajes['Dia_Retiro_Digito'] = df_viajes['Fecha_Retiro_DateTime'].dt.day


In [24]:
# Datos temporales: Ciclo estaciones de arribo
df_viajes['Hora_Arribo_Digito'] = df_viajes['Fecha_Arribo_DateTime'].dt.hour
df_viajes['Dia_Semana_Arribo'] = df_viajes['Fecha_Arribo_DateTime'].dt.day_name()
df_viajes['Dia_Arribo_Digito'] = df_viajes['Fecha_Arribo_DateTime'].dt.day


### Ciclo estaciones

#### Validando ciclo estaciones válidas

In [25]:
# Creando un conjunto de ciclo estaciones válidas

ciclo_estaciones_validas = set(df_ciclo_estaciones['short_name'].astype(str))


In [26]:
'''
Función para validar si una ciclo estación se
encuentra dentro del conjunto de ciclo estaciones válidas
'''

def ciclo_estacion_valida(ciclo_estacion):
    return str(ciclo_estacion) in ciclo_estaciones_validas

In [27]:
# Validando 'Ciclo_Estacion_Retiro' | 'Ciclo_Estacion_Arribo'

df_viajes['Retiro_Valido'] = df_viajes['Ciclo_Estacion_Retiro'].apply(ciclo_estacion_valida)

df_viajes['Arribo_Valido'] = df_viajes['Ciclo_Estacion_Arribo'].apply(ciclo_estacion_valida)

df_viajes.head(2)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,...,Duracion_viaje_minutos,Duracion_viaje,Hora_Retiro_Digito,Dia_Semana_Retiro,Dia_Retiro_Digito,Hora_Arribo_Digito,Dia_Semana_Arribo,Dia_Arribo_Digito,Retiro_Valido,Arribo_Valido
44,O,53.0,2199032,331,01/08/2024,00:01:50,312,01/08/2024,00:04:26,2024-08-01 00:01:50,...,2.6,02 minutos 36 segundos,0,Thursday,1,0,Thursday,1,True,True
51,F,20.0,3530268,104,01/08/2024,00:00:06,055,01/08/2024,00:05:00,2024-08-01 00:00:06,...,4.9,04 minutos 54 segundos,0,Thursday,1,0,Thursday,1,True,True


### Filtros

#### Ciclo estaciones válidas

In [28]:
# Viajes con ciclo estaciones válidas

df_viajes_ciclo_estaciones_validas = df_viajes[df_viajes['Retiro_Valido'] & df_viajes['Arribo_Valido']]

df_viajes_ciclo_estaciones_validas.head(2)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,...,Duracion_viaje_minutos,Duracion_viaje,Hora_Retiro_Digito,Dia_Semana_Retiro,Dia_Retiro_Digito,Hora_Arribo_Digito,Dia_Semana_Arribo,Dia_Arribo_Digito,Retiro_Valido,Arribo_Valido
44,O,53.0,2199032,331,01/08/2024,00:01:50,312,01/08/2024,00:04:26,2024-08-01 00:01:50,...,2.6,02 minutos 36 segundos,0,Thursday,1,0,Thursday,1,True,True
51,F,20.0,3530268,104,01/08/2024,00:00:06,055,01/08/2024,00:05:00,2024-08-01 00:00:06,...,4.9,04 minutos 54 segundos,0,Thursday,1,0,Thursday,1,True,True


In [29]:
print(f"Número total de viajes: {len(df_viajes):,}")
print(f"Número total de viajes con estaciones válidas: {len(df_viajes_ciclo_estaciones_validas):,}")
print(f"Porcentaje de viajes válidos: {len(df_viajes_ciclo_estaciones_validas)/len(df_viajes) * 100}")

Número total de viajes: 1,842,956
Número total de viajes con estaciones válidas: 1,842,901
Porcentaje de viajes válidos: 99.99701566396594


In [30]:
print("Ciclo estaciones de arribo no válidas más comunes:")
print(df_viajes[~df_viajes['Arribo_Valido']]['Ciclo_Estacion_Arribo'].value_counts().head())


Ciclo estaciones de arribo no válidas más comunes:
Ciclo_Estacion_Arribo
1000     54
tag 2     1
Name: count, dtype: int64


In [31]:
print("Ciclo estaciones de retiro no válidas más comunes:")
print(df_viajes[~df_viajes['Retiro_Valido']]['Ciclo_Estacion_Retiro'].value_counts().head())


Ciclo estaciones de retiro no válidas más comunes:
Ciclo_Estacion_Retiro
1000    3
Name: count, dtype: int64


#### Añadiendo clusters

In [32]:
# Retiro
df_viajes_ciclo_estaciones_validas_cluster_retiro = pd.merge(
    df_viajes_ciclo_estaciones_validas,
    df_ciclo_estaciones[['short_name', 'cluster']],
    left_on='Ciclo_Estacion_Retiro',
    right_on='short_name',
    how='left'
)

df_viajes_ciclo_estaciones_validas_cluster_retiro = df_viajes_ciclo_estaciones_validas_cluster_retiro.rename(
    columns={'cluster': 'Cluster_Retiro'}
)

df_viajes_ciclo_estaciones_validas_cluster_retiro = df_viajes_ciclo_estaciones_validas_cluster_retiro.drop('short_name', axis=1)

# Arribo

df_viajes_ciclo_estaciones_validas_cluster = pd.merge(
    df_viajes_ciclo_estaciones_validas_cluster_retiro,
    df_ciclo_estaciones[['short_name', 'cluster']],
    left_on='Ciclo_Estacion_Arribo',
    right_on='short_name',
    how='left'
)

df_viajes_ciclo_estaciones_validas_cluster = df_viajes_ciclo_estaciones_validas_cluster.rename(
    columns={'cluster': 'Cluster_Arribo'}
)

df_viajes_ciclo_estaciones_validas_cluster = df_viajes_ciclo_estaciones_validas_cluster.drop('short_name', axis=1)

df_viajes_ciclo_estaciones_validas_cluster


,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_DateTime,...,Hora_Retiro_Digito,Dia_Semana_Retiro,Dia_Retiro_Digito,Hora_Arribo_Digito,Dia_Semana_Arribo,Dia_Arribo_Digito,Retiro_Valido,Arribo_Valido,Cluster_Retiro,Cluster_Arribo
0,O,53.0,2199032,331,01/08/2024,00:01:50,312,01/08/2024,00:04:26,2024-08-01 00:01:50,...,0,Thursday,1,0,Thursday,1,True,True,3,3
1,F,20.0,3530268,104,01/08/2024,00:00:06,055,01/08/2024,00:05:00,2024-08-01 00:00:06,...,0,Thursday,1,0,Thursday,1,True,True,4,4
2,M,32.0,5796800,008,01/08/2024,00:00:47,026,01/08/2024,00:05:22,2024-08-01 00:00:47,...,0,Thursday,1,0,Thursday,1,True,True,4,4
3,M,32.0,2354306,077,01/08/2024,00:02:05,068,01/08/2024,00:05:27,2024-08-01 00:02:05,...,0,Thursday,1,0,Thursday,1,True,True,0,0
4,M,37.0,7424112,054,01/08/2024,00:01:28,154,01/08/2024,00:05:52,2024-08-01 00:01:28,...,0,Thursday,1,0,Thursday,1,True,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842896,F,28.0,2815322,068,30/08/2024,22:14:49,078,31/08/2024,07:49:36,2024-08-30 22:14:49,...,22,Friday,30,7,Saturday,31,True,True,0,0
1842897,M,65.0,5808704,047,30/08/2024,14:27:32,133,31/08/2024,10:00:09,2024-08-30 14:27:32,...,14,Friday,30,10,Saturday,31,True,True,0,4
1842898,M,52.0,4843876,389,30/08/2024,23:55:19,660,31/08/2024,10:30:14,2024-08-30 23:55:19,...,23,Friday,30,10,Saturday,31,True,True,3,3
1842899,M,22.0,4882494,179,30/08/2024,17:05:42,258,31/08/2024,17:23:14,2024-08-30 17:05:42,...,17,Friday,30,17,Saturday,31,True,True,0,0


#### Eliminación de nulos

In [33]:
# Validando nulos
df_viajes_ciclo_estaciones_validas_cluster.isnull().sum()

Genero_Usuario            15
Edad_Usuario              19
Bici                       0
Ciclo_Estacion_Retiro      0
Fecha_Retiro               0
Hora_Retiro                0
Ciclo_Estacion_Arribo      0
Fecha_Arribo               0
Hora_Arribo                0
Fecha_Retiro_DateTime      0
Fecha_Arribo_DateTime      0
Grupo_Edad                19
Duracion_viaje_minutos     0
Duracion_viaje             0
Hora_Retiro_Digito         0
Dia_Semana_Retiro          0
Dia_Retiro_Digito          0
Hora_Arribo_Digito         0
Dia_Semana_Arribo          0
Dia_Arribo_Digito          0
Retiro_Valido              0
Arribo_Valido              0
Cluster_Retiro             0
Cluster_Arribo             0
dtype: int64

In [34]:
# Eliminando nulos

df_viajes_limpio = df_viajes_ciclo_estaciones_validas_cluster.dropna()

In [35]:
df_viajes_limpio.isnull().sum()

Genero_Usuario            0
Edad_Usuario              0
Bici                      0
Ciclo_Estacion_Retiro     0
Fecha_Retiro              0
Hora_Retiro               0
Ciclo_Estacion_Arribo     0
Fecha_Arribo              0
Hora_Arribo               0
Fecha_Retiro_DateTime     0
Fecha_Arribo_DateTime     0
Grupo_Edad                0
Duracion_viaje_minutos    0
Duracion_viaje            0
Hora_Retiro_Digito        0
Dia_Semana_Retiro         0
Dia_Retiro_Digito         0
Hora_Arribo_Digito        0
Dia_Semana_Arribo         0
Dia_Arribo_Digito         0
Retiro_Valido             0
Arribo_Valido             0
Cluster_Retiro            0
Cluster_Arribo            0
dtype: int64

In [36]:
# Eliminando columnas no necesarias
df_viajes_depurado = df_viajes_limpio.drop(columns=[
    'Fecha_Retiro',
    'Hora_Retiro',
    'Fecha_Arribo',
    'Hora_Arribo',
    'Retiro_Valido',
    'Arribo_Valido'
])


In [37]:
print(f"Total viajes: {len(df_viajes_depurado):,}")

Total viajes: 1,842,882


In [38]:
df_viajes_depurado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1842882 entries, 0 to 1842900
Data columns (total 18 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Genero_Usuario          object        
 1   Edad_Usuario            float64       
 2   Bici                    int64         
 3   Ciclo_Estacion_Retiro   object        
 4   Ciclo_Estacion_Arribo   object        
 5   Fecha_Retiro_DateTime   datetime64[ns]
 6   Fecha_Arribo_DateTime   datetime64[ns]
 7   Grupo_Edad              category      
 8   Duracion_viaje_minutos  float64       
 9   Duracion_viaje          object        
 10  Hora_Retiro_Digito      int32         
 11  Dia_Semana_Retiro       object        
 12  Dia_Retiro_Digito       int32         
 13  Hora_Arribo_Digito      int32         
 14  Dia_Semana_Arribo       object        
 15  Dia_Arribo_Digito       int32         
 16  Cluster_Retiro          int64         
 17  Cluster_Arribo          int64         
dtypes: cate

## Exportando viajes válidos en formato CSV

In [39]:
df_viajes_depurado.to_csv('viajes_data_limpia.csv', index=False)